### Data Upload and Filtering

In [174]:
import pandas as pd

# Load the data from the Excel file
file_path = 'C:/Users/olcay/Desktop/TASKS/TrakRacer/AUS Shipping Comparison/Take a look later on/Order_AUS.xlsx'
data = pd.read_excel(file_path)


In [101]:
# Define a list of pickup-related shipping methods to exclude
pickup_methods = [
    'free', 'freee', 'Free shipping', 
    'Pick-up at Warehouse: Hallam Valley Rd 47, 3175 Melbourne (AU)', 
    'Pick up'
]

# Identifying all Names with pickup-related shipping methods
names_to_exclude = data[data['Shipping Method'].isin(pickup_methods)]['Name'].unique()

# Filtering out orders with these Names
data_filtered_excluding_pickup = data[~data['Name'].isin(names_to_exclude)]

# Selecting relevant columns for analysis
data_filtered_excluding_pickup = data_filtered_excluding_pickup[['Name', 'Lineitem sku', 'Lineitem price', 'Lineitem quantity', 'Discount Amount', 'Shipping', 'Shipping Province Name', 'Total']]

# Calculate subtotal per order by multiplying Lineitem price by quantity
data_filtered_excluding_pickup['subtotal'] = data_filtered_excluding_pickup['Lineitem price'] * data_filtered_excluding_pickup['Lineitem quantity']

# Group by 'Name' to sum subtotals and other relevant fields
grouped_data_excluding_pickup = data_filtered_excluding_pickup.groupby('Name').agg(
    subtotal=('subtotal', 'sum'),
    discount=('Discount Amount', 'sum'),
    shipping=('Shipping', lambda x: x.dropna().iloc[0] if not x.dropna().empty else pd.NA),
    original_total=('Total', lambda x: x.dropna().iloc[0] if not x.dropna().empty else pd.NA),
    province=('Shipping Province Name', lambda x: x.dropna().iloc[0] if not x.dropna().empty else 'Unknown')
).reset_index()

# Verify if `TRAU48683` and other such entries have been removed
grouped_data_excluding_pickup[grouped_data_excluding_pickup['Name'] == 'TRAU48683'] # example of a pickup or something

# Create a copy to avoid SettingWithCopyWarning
grouped_data = grouped_data_excluding_pickup[grouped_data_excluding_pickup['shipping'] != 0].copy()

# Calculate the adjusted subtotal after applying the discount
grouped_data['adjusted_subtotal'] = grouped_data['subtotal'] - grouped_data['discount']

# Assume all costs include a 10% GST
gst_rate = 0.10

# Remove GST from subtotal and shipping to get base amounts
grouped_data['base_subtotal'] = grouped_data['subtotal'] / (1 + gst_rate)
grouped_data['base_shipping'] = grouped_data['shipping'] / (1 + gst_rate)

# Recalculate the adjusted subtotal if necessary
grouped_data['adjusted_subtotal'] = grouped_data['base_subtotal'] - grouped_data['discount']

# Calculate the required increase percentage to cover base shipping costs
grouped_data['required_increase_percentage'] = (grouped_data['base_shipping'] / grouped_data['adjusted_subtotal']) * 100




In [102]:
# Including the Australian Capital Territory in the free shipping areas list
free_shipping_areas = ['Victoria', 'New South Wales', 'Queensland', 'Australian Capital Territory', 'South Australia']

# Filter the data for the updated free shipping areas and non free shipping areas
free_shipping_data_updated = grouped_data[grouped_data['province'].isin(free_shipping_areas)]
non_free_shipping_areas = grouped_data[~grouped_data['province'].isin(free_shipping_areas)]


In [103]:
# Totals for free shipping areas
total_base_shipping_free_shipping_areas = free_shipping_data_updated['base_shipping'].sum()
total_base_subtotal_free_shipping_areas = free_shipping_data_updated['base_subtotal'].sum()
total_base_shipping_non_free_shipping_areas = non_free_shipping_areas['base_shipping'].sum()
total_base_subtotal_non_free_shipping_areas = non_free_shipping_areas['base_subtotal'].sum()

### Validation TRY

In [ ]:
import pandas as pd

# Load the data from the Excel file
file_path = 'C:/Users/olcay/Desktop/TASKS/TrakRacer/AUS Shipping Comparison/Take a look later on/Order_AUS.xlsx'
data = pd.read_excel(file_path)

# Define a list of pickup-related shipping methods to exclude
pickup_methods = [
    'free', 'freee', 'Free shipping', 
    'Pick-up at Warehouse: Hallam Valley Rd 47, 3175 Melbourne (AU)', 
    'Pick up'
]

# Identifying all Names with pickup-related shipping methods
names_to_exclude = data[data['Shipping Method'].isin(pickup_methods)]['Name'].unique()

# Filtering out orders with these Names
data_filtered_excluding_pickup = data[~data['Name'].isin(names_to_exclude)]

# Selecting relevant columns for analysis
data_filtered_excluding_pickup = data_filtered_excluding_pickup[['Name', 'Lineitem sku', 'Lineitem price', 'Lineitem quantity', 'Discount Amount', 'Shipping', 'Shipping Province Name', 'Total']]

# Calculate subtotal per order by multiplying Lineitem price by quantity
data_filtered_excluding_pickup['subtotal'] = data_filtered_excluding_pickup['Lineitem price'] * data_filtered_excluding_pickup['Lineitem quantity']

# Group by 'Name' to sum subtotals and other relevant fields
grouped_data_excluding_pickup = data_filtered_excluding_pickup.groupby('Name').agg(
    subtotal=('subtotal', 'sum'),
    discount=('Discount Amount', 'sum'),
    shipping=('Shipping', lambda x: x.dropna().iloc[0] if not x.dropna().empty else pd.NA),
    original_total=('Total', lambda x: x.dropna().iloc[0] if not x.dropna().empty else pd.NA),
    province=('Shipping Province Name', lambda x: x.dropna().iloc[0] if not x.dropna().empty else 'Unknown')
).reset_index()

# Verify if `TRAU48683` and other such entries have been removed
grouped_data_excluding_pickup[grouped_data_excluding_pickup['Name'] == 'TRAU48683']

# Create a copy to avoid SettingWithCopyWarning
grouped_data = grouped_data_excluding_pickup[grouped_data_excluding_pickup['shipping'] != 0].copy()

# Calculate the adjusted subtotal after applying the discount
grouped_data['adjusted_subtotal'] = grouped_data['subtotal'] - grouped_data['discount']

# Assume all costs include a 10% GST
gst_rate = 0.10

# Remove GST from subtotal and shipping to get base amounts
grouped_data['base_subtotal'] = grouped_data['subtotal'] / (1 + gst_rate)
grouped_data['base_shipping'] = grouped_data['shipping'] / (1 + gst_rate)

# Define a list of free shipping areas
free_shipping_areas = ['Victoria', 'New South Wales', 'Queensland', 'Australian Capital Territory', 'South Australia']

# Classify orders into free and non-free shipping zones
grouped_data['is_free_shipping_zone'] = grouped_data['province'].isin(free_shipping_areas)

# Determine if the orders are above the minimum order value for free shipping
min_order_value = 100  # Example value, adjust as needed
grouped_data['above_min_order_value'] = grouped_data['base_subtotal'] >= min_order_value

# Calculate the final shipping cost considering the free shipping policy
grouped_data['final_shipping_cost'] = grouped_data.apply(
    lambda x: 0 if (x['is_free_shipping_zone'] and x['above_min_order_value']) else x['base_shipping'], axis=1
)

# Calculate the net coverage per item
grouped_data['net_coverage_per_item'] = grouped_data['additional_revenue_corrected'] - grouped_data['final_shipping_cost']

# Define the percentages and minimum order values for analysis
price_increase_percentages = [0.01, 0.02, 0.03, 0.05, 0.07, 0.10]
min_order_values = [50, 100, 150, 200, 300, 400, 500, 750, 1000]

results_list = []

for percentage in price_increase_percentages:
    for min_order_value in min_order_values:
        # Calculate the corrected increased prices and additional revenue for the current percentage
        grouped_data['correct_increased_price'] = grouped_data['subtotal'] * (1 + percentage)
        grouped_data['additional_revenue_corrected'] = grouped_data['correct_increased_price'] - grouped_data['subtotal']
        
        # Separate free and non-free shipping areas again
        free_shipping_data_corrected = grouped_data[grouped_data['is_free_shipping_zone']]
        non_free_shipping_areas_corrected = grouped_data[~grouped_data['is_free_shipping_zone']]
        
        # Orders above the minimum order value for free shipping in free shipping areas
        orders_above_minimum_value_corrected = free_shipping_data_corrected[free_shipping_data_corrected['base_subtotal'] >= min_order_value]
        total_shipping_cost_above_minimum_value_corrected = orders_above_minimum_value_corrected['base_shipping'].sum()
        
        # Calculate the total additional revenue
        total_additional_revenue_corrected = grouped_data['additional_revenue_corrected'].sum()
        
        # Calculate the net difference for free shipping zones
        net_difference_free_shipping_corrected = total_additional_revenue_corrected - total_shipping_cost_above_minimum_value_corrected
        
        # Net difference for non-free shipping areas
        total_additional_revenue_non_free_corrected = non_free_shipping_areas_corrected['additional_revenue_corrected'].sum()
        net_difference_non_free_shipping_corrected = total_additional_revenue_non_free_corrected
        
        # Overall net difference
        overall_net_difference_final = net_difference_free_shipping_corrected + net_difference_non_free_shipping_corrected
        
        # Store the results
        results_list.append({
            "percentage_increase": percentage,
            "min_order_value": min_order_value,
            "total_additional_revenue_corrected": total_additional_revenue_corrected,
            "total_shipping_cost_above_minimum_value_corrected": total_shipping_cost_above_minimum_value_corrected,
            "net_difference_free_shipping_corrected": net_difference_free_shipping_corrected,
            "net_difference_non_free_shipping_corrected": net_difference_non_free_shipping_corrected,
            "overall_net_difference_final": overall_net_difference_final
        })

# Convert the results to a DataFrame for easy viewing and save to an Excel file
# results_df = pd.DataFrame(results_list)
# results_df.to_excel('price_increase_analysis.xlsx', index=False)


### Optimization Strategy

In [160]:
# Initial setup
initial_increase_percentage = 6
adjustment_step = 0.05
minimum_order_value_for_free_shipping = 50  # Start with an example value
max_iterations = 100  # To prevent excessive computation time

In [161]:
# Define a function to calculate the net difference with the given parameters
def calculate_net_difference_with_non_free(increase_percentage, min_order_value):
    # Orders below the minimum order value for free shipping in free shipping areas
    orders_below_minimum_value = free_shipping_data_updated[free_shipping_data_updated['base_subtotal'] < min_order_value]
    total_shipping_cost_below_minimum_value = orders_below_minimum_value['base_shipping'].sum()
    
    # Total base shipping costs including non-free areas
    total_base_shipping_including_minimum_value_orders = (
        total_base_shipping_free_shipping_areas +
        total_shipping_cost_below_minimum_value +
        total_base_shipping_non_free_shipping_areas
    )
    
    # Additional revenue from price increase
    total_base_subtotal_including_non_free = total_base_subtotal_free_shipping_areas + total_base_subtotal_non_free_shipping_areas
    additional_revenue_from_price_increase = total_base_subtotal_including_non_free * (increase_percentage / 100)
    
    # Adjust shipping cost reduction for non-free areas
    reduced_shipping_cost_non_free_areas = total_base_shipping_non_free_shipping_areas * (1 - (increase_percentage / 100))
    total_base_shipping_including_minimum_value_orders -= reduced_shipping_cost_non_free_areas
    
    net_difference = additional_revenue_from_price_increase - total_base_shipping_including_minimum_value_orders
    
    return net_difference, additional_revenue_from_price_increase, total_shipping_cost_below_minimum_value

# Loop to adjust the minimum order value and find the optimal point
iteration_count = 0
while iteration_count < max_iterations:
    net_difference, additional_revenue, total_shipping_cost_below_minimum_value = calculate_net_difference_with_non_free(
        initial_increase_percentage, minimum_order_value_for_free_shipping)
    
    if net_difference >= 0:
        break  # Exit the loop if we reach a net-zero or positive difference
    
    minimum_order_value_for_free_shipping += 25  # Increment the minimum order value
    iteration_count += 1

# Output the results for the optimal minimum order value and price increase
optimal_results_with_non_free = {
    "minimum_order_value_for_free_shipping": minimum_order_value_for_free_shipping,
    "initial_increase_percentage": initial_increase_percentage,
    "additional_revenue": additional_revenue,
    "net_difference": net_difference,
    "total_shipping_cost_below_minimum_value": total_shipping_cost_below_minimum_value
}



In [162]:
optimal_results_with_non_free

{'minimum_order_value_for_free_shipping': 50,
 'initial_increase_percentage': 6,
 'additional_revenue': np.float64(26174.050909090904),
 'net_difference': np.float64(1089.0870909090881),
 'total_shipping_cost_below_minimum_value': np.float64(690.9272727272727)}

In [163]:
# Define a function to calculate the net difference with the given parameters
def calculate_net_difference_with_non_free(increase_percentage, min_order_value):
    # Orders below the minimum order value for free shipping in free shipping areas
    orders_below_minimum_value = free_shipping_data_updated[free_shipping_data_updated['base_subtotal'] < min_order_value]
    total_shipping_cost_below_minimum_value = orders_below_minimum_value['base_shipping'].sum()
    
    # Additional revenue from price increase
    total_base_subtotal_including_non_free = total_base_subtotal_free_shipping_areas + total_base_subtotal_non_free_shipping_areas
    additional_revenue_from_price_increase = total_base_subtotal_including_non_free * (increase_percentage / 100)
    
    # The net difference should only deduct the shipping costs for orders below the minimum order value in free shipping zones
    net_difference = additional_revenue_from_price_increase - total_shipping_cost_below_minimum_value
    
    return net_difference, additional_revenue_from_price_increase, total_shipping_cost_below_minimum_value

# Loop to adjust the minimum order value and find the optimal point
iteration_count = 0
max_iterations = 100
initial_increase_percentage = 6.00
minimum_order_value_for_free_shipping = 100  # Initial value

while iteration_count < max_iterations:
    net_difference, additional_revenue, total_shipping_cost_below_minimum_value = calculate_net_difference_with_non_free(
        initial_increase_percentage, minimum_order_value_for_free_shipping)
    
    if net_difference >= 0:
        break  # Exit the loop if we reach a net-zero or positive difference
    
    minimum_order_value_for_free_shipping += 25  # Increment the minimum order value
    iteration_count += 1

# Output the results for the optimal minimum order value and price increase
optimal_results_with_non_free = {
    "minimum_order_value_for_free_shipping": minimum_order_value_for_free_shipping,
    "initial_increase_percentage": initial_increase_percentage,
    "additional_revenue": additional_revenue,
    "net_difference": net_difference,
    "total_shipping_cost_below_minimum_value": total_shipping_cost_below_minimum_value
}



In [164]:
optimal_results_with_non_free

{'minimum_order_value_for_free_shipping': 100,
 'initial_increase_percentage': 6.0,
 'additional_revenue': np.float64(26174.050909090904),
 'net_difference': np.float64(24576.050909090904),
 'total_shipping_cost_below_minimum_value': np.float64(1598.0)}

### Once Again

In [50]:
# Initial price increase percentage
initial_increase_percentage = 6.85
adjustment_step = 0.05

In [51]:
# Assuming gst_rate and data have been defined and loaded
gst_rate = 0.10

# Base calculations excluding GST
grouped_data['base_subtotal'] = grouped_data['subtotal'] / (1 + gst_rate)
grouped_data['base_shipping'] = grouped_data['shipping'] / (1 + gst_rate)

# Calculate the total base shipping cost and base subtotal for free shipping areas
free_shipping_data_updated = grouped_data[grouped_data['province'].isin(free_shipping_areas)]
total_base_shipping_free_shipping_areas = free_shipping_data_updated['base_shipping'].sum()
total_base_subtotal_free_shipping_areas = free_shipping_data_updated['base_subtotal'].sum()

# Initial price increase percentage
initial_increase_percentage = 6.00
adjustment_step = 0.05

# Calculate additional revenue from the initial price increase
additional_revenue_from_price_increase = total_base_subtotal_free_shipping_areas * (initial_increase_percentage / 100)

# Calculate the net difference
net_difference = additional_revenue_from_price_increase - total_base_shipping_free_shipping_areas

# Adjust the percentage to achieve a net-zero or positive difference
while net_difference < 0:
    initial_increase_percentage += adjustment_step
    additional_revenue_from_price_increase = total_base_subtotal_free_shipping_areas * (initial_increase_percentage / 100)
    net_difference = additional_revenue_from_price_increase - total_base_shipping_free_shipping_areas

# Output results
print(f"Optimal Price Increase Percentage: {initial_increase_percentage:.2f}%")
print(f"Additional Revenue from Price Increase: AUD {additional_revenue_from_price_increase:.2f}")
print(f"Net Difference: AUD {net_difference:.2f}")

# Reduction in non-free shipping areas
non_free_shipping_areas = grouped_data[~grouped_data['province'].isin(free_shipping_areas)]
total_base_shipping_non_free_shipping_areas = non_free_shipping_areas['base_shipping'].sum()
reduction_percentage = initial_increase_percentage
reduced_base_shipping_non_free_shipping_areas = total_base_shipping_non_free_shipping_areas * (1 - reduction_percentage / 100)
savings_from_shipping_cost_reduction = total_base_shipping_non_free_shipping_areas - reduced_base_shipping_non_free_shipping_areas

print(f"Reduction in Shipping Costs for Non-Free Shipping Areas: {reduction_percentage:.2f}%")
print(f"Savings from Shipping Cost Reduction: AUD {savings_from_shipping_cost_reduction:.2f}")

# Minimum order value for free shipping
average_base_shipping_cost_per_order = total_base_shipping_free_shipping_areas / len(free_shipping_data_updated)
minimum_order_value_for_free_shipping = average_base_shipping_cost_per_order / (initial_increase_percentage / 100)
print(f"Minimum Order Value for Free Shipping: AUD {minimum_order_value_for_free_shipping:.2f}")


Optimal Price Increase Percentage: 6.70%
Additional Revenue from Price Increase: AUD 24117.00
Net Difference: AUD 48.26
Reduction in Shipping Costs for Non-Free Shipping Areas: 6.70%
Savings from Shipping Cost Reduction: AUD 363.24
Minimum Order Value for Free Shipping: AUD 694.85


In [68]:
# Define minimum order value for free shipping
minimum_order_value_for_free_shipping = 694  # Example value

# Identify orders below the minimum order value
orders_below_minimum_value = free_shipping_data_updated[free_shipping_data_updated['base_subtotal'] < minimum_order_value_for_free_shipping]

# Calculate total shipping cost for these orders
total_shipping_cost_below_minimum_value = orders_below_minimum_value['base_shipping'].sum()

# Ensure these costs are included in the total base shipping costs
total_base_shipping_free_shipping_areas += total_shipping_cost_below_minimum_value

# Calculate net difference with updated total shipping costs
net_difference = additional_revenue_from_price_increase - total_base_shipping_free_shipping_areas


In [69]:
net_difference

np.float64(-60355.75127272727)

In [83]:
# Base calculations excluding GST
gst_rate = 0.10
grouped_data['base_subtotal'] = grouped_data['subtotal'] / (1 + gst_rate)
grouped_data['base_shipping'] = grouped_data['shipping'] / (1 + gst_rate)

# Separate orders into free and non-free shipping areas
free_shipping_data_updated = grouped_data[grouped_data['province'].isin(free_shipping_areas)]
non_free_shipping_areas = grouped_data[~grouped_data['province'].isin(free_shipping_areas)]

# Define an initial minimum order value for free shipping
initial_minimum_order_value_for_free_shipping = 100  # Initial example value

# Identify orders below the initial minimum order value
orders_below_minimum_value = free_shipping_data_updated[free_shipping_data_updated['base_subtotal'] < initial_minimum_order_value_for_free_shipping]

# Calculate total shipping cost for these orders
total_shipping_cost_below_minimum_value = orders_below_minimum_value['base_shipping'].sum()

# Calculate the total base shipping cost including orders below the minimum value
total_base_shipping_free_shipping_areas = free_shipping_data_updated['base_shipping'].sum()
total_base_shipping_including_minimum_value_orders = total_base_shipping_free_shipping_areas + total_shipping_cost_below_minimum_value

# Initial price increase percentage
initial_increase_percentage = 6.00
adjustment_step = 0.05

# Calculate additional revenue from the initial price increase
total_base_subtotal_free_shipping_areas = free_shipping_data_updated['base_subtotal'].sum()
additional_revenue_from_price_increase = total_base_subtotal_free_shipping_areas * (initial_increase_percentage / 100)

# Calculate the net difference
net_difference = additional_revenue_from_price_increase - total_base_shipping_including_minimum_value_orders

# Adjust the percentage to achieve a net-zero or positive difference
while net_difference < 0:
    initial_increase_percentage += adjustment_step
    additional_revenue_from_price_increase = total_base_subtotal_free_shipping_areas * (initial_increase_percentage / 100)
    net_difference = additional_revenue_from_price_increase - total_base_shipping_including_minimum_value_orders

# Output results for the optimal price increase
print(f"Optimal Price Increase Percentage: {initial_increase_percentage:.2f}%")
print(f"Additional Revenue from Price Increase: AUD {additional_revenue_from_price_increase:.2f}")
print(f"Net Difference: AUD {net_difference:.2f}")

# Calculate the reduction in shipping costs for non-free shipping areas
total_base_shipping_non_free_shipping_areas = non_free_shipping_areas['base_shipping'].sum()
reduction_percentage = initial_increase_percentage
reduced_base_shipping_non_free_shipping_areas = total_base_shipping_non_free_shipping_areas * (1 - reduction_percentage / 100)
savings_from_shipping_cost_reduction = total_base_shipping_non_free_shipping_areas - reduced_base_shipping_non_free_shipping_areas

print(f"Reduction in Shipping Costs for Non-Free Shipping Areas: {reduction_percentage:.2f}%")
print(f"Savings from Shipping Cost Reduction: AUD {savings_from_shipping_cost_reduction:.2f}")

# Determine the minimum order value for free shipping that covers the shipping costs
average_base_shipping_cost_per_order = total_base_shipping_free_shipping_areas / len(free_shipping_data_updated)
minimum_order_value_for_free_shipping = average_base_shipping_cost_per_order / (initial_increase_percentage / 100)

print(f"Minimum Order Value for Free Shipping: AUD {minimum_order_value_for_free_shipping:.2f}")


Optimal Price Increase Percentage: 7.15%
Additional Revenue from Price Increase: AUD 25736.80
Net Difference: AUD 70.06
Reduction in Shipping Costs for Non-Free Shipping Areas: 7.15%
Savings from Shipping Cost Reduction: AUD 387.64
Minimum Order Value for Free Shipping: AUD 651.11


In [82]:
# Base calculations excluding GST
gst_rate = 0.10
grouped_data['base_subtotal'] = grouped_data['subtotal'] / (1 + gst_rate)
grouped_data['base_shipping'] = grouped_data['shipping'] / (1 + gst_rate)

# Separate orders into free and non-free shipping areas
free_shipping_data_updated = grouped_data[grouped_data['province'].isin(free_shipping_areas)]
non_free_shipping_areas = grouped_data[~grouped_data['province'].isin(free_shipping_areas)]

# Define minimum order value for free shipping
minimum_order_value_for_free_shipping = 100  # Example value

# Identify orders below the minimum order value
orders_below_minimum_value = free_shipping_data_updated[free_shipping_data_updated['base_subtotal'] < minimum_order_value_for_free_shipping]

# Calculate total shipping cost for these orders
total_shipping_cost_below_minimum_value = orders_below_minimum_value['base_shipping'].sum()

# Calculate the total base shipping cost including orders below the minimum value
total_base_shipping_free_shipping_areas = free_shipping_data_updated['base_shipping'].sum()
total_base_shipping_including_minimum_value_orders = total_base_shipping_free_shipping_areas + total_shipping_cost_below_minimum_value

# Initial price increase percentage
initial_increase_percentage = 6.00
adjustment_step = 0.05

# Calculate additional revenue from the initial price increase
total_base_subtotal_free_shipping_areas = free_shipping_data_updated['base_subtotal'].sum()
additional_revenue_from_price_increase = total_base_subtotal_free_shipping_areas * (initial_increase_percentage / 100)

# Calculate the net difference
net_difference = additional_revenue_from_price_increase - total_base_shipping_including_minimum_value_orders

# Adjust the percentage to achieve a net-zero or positive difference
while net_difference < 0:
    initial_increase_percentage += adjustment_step
    additional_revenue_from_price_increase = total_base_subtotal_free_shipping_areas * (initial_increase_percentage / 100)
    net_difference = additional_revenue_from_price_increase - total_base_shipping_including_minimum_value_orders

# Output results
print(f"Optimal Price Increase Percentage: {initial_increase_percentage:.2f}%")
print(f"Additional Revenue from Price Increase: AUD {additional_revenue_from_price_increase:.2f}")
print(f"Net Difference: AUD {net_difference:.2f}")

# Reduction in shipping costs for non-free shipping areas
total_base_shipping_non_free_shipping_areas = non_free_shipping_areas['base_shipping'].sum()
reduction_percentage = initial_increase_percentage
reduced_base_shipping_non_free_shipping_areas = total_base_shipping_non_free_shipping_areas * (1 - reduction_percentage / 100)
savings_from_shipping_cost_reduction = total_base_shipping_non_free_shipping_areas - reduced_base_shipping_non_free_shipping_areas

print(f"Reduction in Shipping Costs for Non-Free Shipping Areas: {reduction_percentage:.2f}%")
print(f"Savings from Shipping Cost Reduction: AUD {savings_from_shipping_cost_reduction:.2f}")


Optimal Price Increase Percentage: 7.15%
Additional Revenue from Price Increase: AUD 25736.80
Net Difference: AUD 70.06
Reduction in Shipping Costs for Non-Free Shipping Areas: 7.15%
Savings from Shipping Cost Reduction: AUD 387.64


# Validation

In [84]:
# Define the optimal price increase percentage
optimal_increase_percentage = 7.15

# Define the minimum order value for free shipping
minimum_order_value_for_free_shipping = 651.11  # Example value

# Calculate the adjusted subtotal for all orders after the price increase
grouped_data['adjusted_subtotal'] = grouped_data['base_subtotal'] * (1 + optimal_increase_percentage / 100)

# Filter orders to find those above the minimum order value threshold after the price increase
orders_above_threshold = grouped_data[grouped_data['adjusted_subtotal'] >= minimum_order_value_for_free_shipping]

# Calculate the total base shipping cost for these orders
total_base_shipping_above_threshold = orders_above_threshold['base_shipping'].sum()

# Calculate the additional revenue from these orders
additional_revenue_above_threshold = orders_above_threshold['adjusted_subtotal'].sum() - orders_above_threshold['base_subtotal'].sum()

# Calculate the net coverage difference
net_coverage_difference = additional_revenue_above_threshold - total_base_shipping_above_threshold

# Total number of orders in the free shipping zone
total_orders_free_zone = len(free_shipping_data_updated)

# Number of orders above the minimum threshold
number_of_orders_above_threshold = len(orders_above_threshold)

# Percentage of orders covered (above the threshold)
percentage_of_orders_covered = (number_of_orders_above_threshold / total_orders_free_zone) * 100

# Output the results
print(f"Total Number of Orders in Free Shipping Zone: {total_orders_free_zone}")
print(f"Number of Orders Above Threshold: {number_of_orders_above_threshold}")
print(f"Percentage of Orders Covered: {percentage_of_orders_covered:.2f}%")
print(f"Total Base Shipping Cost for Orders Above Threshold: AUD {total_base_shipping_above_threshold:.2f}")
print(f"Additional Revenue from Orders Above Threshold: AUD {additional_revenue_above_threshold:.2f}")
print(f"Net Coverage Difference: AUD {net_coverage_difference:.2f}")


Total Number of Orders in Free Shipping Zone: 517
Number of Orders Above Threshold: 233
Percentage of Orders Covered: 45.07%
Total Base Shipping Cost for Orders Above Threshold: AUD 22285.65
Additional Revenue from Orders Above Threshold: AUD 26580.22
Net Coverage Difference: AUD 4294.58


In [86]:
# Initial setup
initial_increase_percentage = 6.00
adjustment_step = 0.05
minimum_order_value_for_free_shipping = 100  # Start with an example value

# Define a function to calculate the net difference with the given parameters
def calculate_net_difference(increase_percentage, min_order_value):
    orders_below_minimum_value = free_shipping_data_updated[free_shipping_data_updated['base_subtotal'] < min_order_value]
    total_shipping_cost_below_minimum_value = orders_below_minimum_value['base_shipping'].sum()
    
    total_base_shipping_including_minimum_value_orders = total_base_shipping_free_shipping_areas + total_shipping_cost_below_minimum_value
    additional_revenue_from_price_increase = total_base_subtotal_free_shipping_areas * (increase_percentage / 100)
    net_difference = additional_revenue_from_price_increase - total_base_shipping_including_minimum_value_orders
    
    return net_difference, additional_revenue_from_price_increase, total_shipping_cost_below_minimum_value

# Loop to adjust the minimum order value and find the optimal point
while True:
    net_difference, additional_revenue, total_shipping_cost_below_minimum_value = calculate_net_difference(initial_increase_percentage, minimum_order_value_for_free_shipping)
    
    if net_difference >= 0:
        break  # Exit the loop if we reach a net-zero or positive difference
    
    minimum_order_value_for_free_shipping += 50  # Increment the minimum order value

# Output the results for the optimal minimum order value and price increase
print(f"Optimal Minimum Order Value for Free Shipping: AUD {minimum_order_value_for_free_shipping:.2f}")
print(f"Optimal Price Increase Percentage: {initial_increase_percentage:.2f}%")
print(f"Additional Revenue from Price Increase: AUD {additional_revenue:.2f}")
print(f"Net Difference: AUD {net_difference:.2f}")
print(f"Total Shipping Cost for Orders Below Minimum Value: AUD {total_shipping_cost_below_minimum_value:.2f}")


KeyboardInterrupt: 

try

In [32]:
# Assuming grouped_data and free_shipping_data_updated are defined and contain the necessary columns

# Set the optimal increase percentage
optimal_increase_percentage = 6.70

# Identify covered and uncovered orders based on the required increase percentage
covered_orders = free_shipping_data_updated[free_shipping_data_updated['required_increase_percentage'] <= optimal_increase_percentage]
uncovered_orders = free_shipping_data_updated[free_shipping_data_updated['required_increase_percentage'] > optimal_increase_percentage]

# Calculate the number of covered and uncovered orders
number_of_covered_orders = len(covered_orders)
number_of_uncovered_orders = len(uncovered_orders)

# Print the results
print(f"Number of Covered Orders: {number_of_covered_orders}")
print(f"Number of Uncovered Orders: {number_of_uncovered_orders}")


Number of Covered Orders: 118
Number of Uncovered Orders: 399


In [36]:
# Verify base subtotal and base shipping cost calculations
total_base_subtotal_free_shipping_areas = free_shipping_data_updated['base_subtotal'].sum()
total_base_shipping_free_shipping_areas = free_shipping_data_updated['base_shipping'].sum()

# Calculate additional revenue from price increase
additional_revenue_from_price_increase = total_base_subtotal_free_shipping_areas * (optimal_increase_percentage / 100)

# Calculate total shipping cost for uncovered orders
total_shipping_cost_uncovered_orders = uncovered_orders['base_shipping'].sum()

# Calculate savings from reduced shipping costs in non-free shipping areas
total_base_shipping_non_free_shipping_areas = non_free_shipping_areas['base_shipping'].sum()
reduced_shipping_cost_non_free_shipping_areas = total_base_shipping_non_free_shipping_areas * (1 - optimal_increase_percentage / 100)
savings_from_shipping_cost_reduction = total_base_shipping_non_free_shipping_areas - reduced_shipping_cost_non_free_shipping_areas

# Calculate net difference
net_difference = additional_revenue_from_price_increase + savings_from_shipping_cost_reduction - total_shipping_cost_uncovered_orders

# Print the results
print(f"Optimal Price Increase Percentage: {optimal_increase_percentage:.2f}%")
print(f"Additional Revenue from Price Increase: AUD {additional_revenue_from_price_increase:.2f}")
print(f"Total Shipping Cost for Uncovered Orders: AUD {total_shipping_cost_uncovered_orders:.2f}")
print(f"Savings from Shipping Cost Reduction: AUD {savings_from_shipping_cost_reduction:.2f}")
print(f"Net Difference: AUD {net_difference:.2f}")


Optimal Price Increase Percentage: 6.70%
Additional Revenue from Price Increase: AUD 24117.00
Total Shipping Cost for Uncovered Orders: AUD 17968.03
Savings from Shipping Cost Reduction: AUD 363.24
Net Difference: AUD 6512.22


### Some Calculations including GST

In [5]:
# Calculate the average shipping cost for the updated free shipping areas
average_shipping_cost_free_shipping_areas_updated = free_shipping_data_updated['shipping'].mean()

average_shipping_cost_free_shipping_areas_updated

np.float64(51.21009671179884)

In [8]:
free_shipping_data_updated.columns

Index(['Name', 'subtotal', 'discount', 'shipping', 'original_total',
       'province', 'adjusted_subtotal', 'required_increase_percentage'],
      dtype='object')

In [10]:
# Calculate the total shipping cost for the current month
total_shipping_cost_current_month = free_shipping_data_updated['shipping'].sum()

# Calculate the total number of units sold
total_units_sold = free_shipping_data_updated['adjusted_subtotal'].sum()

total_original = free_shipping_data_updated['original_total'].sum()

total_shipping_cost_current_month, total_units_sold, total_original


(np.float64(26475.62), np.float64(386557.44), np.float64(413033.06000000006))

In [17]:

# Calculate the required increase percentage for each order to cover shipping costs
free_shipping_data_updated['required_increase_percentage'] = (free_shipping_data_updated['shipping'] / free_shipping_data_updated['adjusted_subtotal']) * 100

# Identify orders that would not be covered by the 6.85% price increase
uncovered_orders = free_shipping_data_updated[free_shipping_data_updated['required_increase_percentage'] > 6.85]

# Calculate how many orders would be covered and not covered
covered_orders_count = len(free_shipping_data_updated) - len(uncovered_orders)
uncovered_orders_count = len(uncovered_orders)

print(covered_orders_count)
print(uncovered_orders_count)

C:\Users\olcay\AppData\Local\Temp\ipykernel_9108\3552656032.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  free_shipping_data_updated['required_increase_percentage'] = (free_shipping_data_updated['shipping'] / free_shipping_data_updated['adjusted_subtotal']) * 100


(np.float64(22.85), 126, 391)

In [34]:
# Calculate the additional revenue from a 6.85% price increase on the covered orders

# Calculate the total adjusted subtotal of the covered orders
covered_orders = free_shipping_data_updated[free_shipping_data_updated['required_increase_percentage'] <= 6.70]
total_covered_orders_subtotal = covered_orders['adjusted_subtotal'].sum()

# Calculate the additional revenue from the price increase
additional_revenue_from_price_increase = total_covered_orders_subtotal * (6.85 / 100)

# Calculate the total shipping cost for uncovered orders
total_shipping_cost_uncovered_orders = uncovered_orders['shipping'].sum()

# Calculate the difference to check if we reach a zero net
net_difference = additional_revenue_from_price_increase - total_shipping_cost_uncovered_orders

print(f"Total Adjusted Subtotal of Covered Orders: AUD {total_covered_orders_subtotal:.2f}")
print(f"Additional Revenue from 6.85% Price Increase: AUD {additional_revenue_from_price_increase:.2f}")
print(f"Total Shipping Cost for Uncovered Orders: AUD {total_shipping_cost_uncovered_orders:.2f}")
print(f"Net Difference: AUD {net_difference:.2f}")

Total Adjusted Subtotal of Covered Orders: AUD 176569.19
Additional Revenue from 6.85% Price Increase: AUD 12094.99
Total Shipping Cost for Uncovered Orders: AUD 19764.83
Net Difference: AUD -7669.84


In [18]:
# Define non-free shipping areas (areas not in the free shipping list)
non_free_shipping_areas = grouped_data[~grouped_data['province'].isin(free_shipping_areas)]

# Calculate the current total shipping cost in non-free shipping areas
total_shipping_cost_non_free_shipping_areas = non_free_shipping_areas['shipping'].sum()

# Apply a 6.85% reduction in shipping costs for non-free shipping areas
reduction_percentage = 6.85
reduced_shipping_cost_non_free_shipping_areas = total_shipping_cost_non_free_shipping_areas * (1 - reduction_percentage / 100)

# Calculate the savings from reducing shipping costs
savings_from_shipping_cost_reduction = total_shipping_cost_non_free_shipping_areas - reduced_shipping_cost_non_free_shipping_areas

# Calculate the net impact considering both the additional revenue from the price increase and the savings from reduced shipping costs
net_impact = additional_revenue_from_price_increase + savings_from_shipping_cost_reduction - total_shipping_cost_uncovered_orders

# Print the results for clarity
print(f"Total Shipping Cost in Non-Free Shipping Areas: AUD {total_shipping_cost_non_free_shipping_areas:.2f}")
print(f"Reduced Shipping Cost in Non-Free Shipping Areas: AUD {reduced_shipping_cost_non_free_shipping_areas:.2f}")
print(f"Savings from Shipping Cost Reduction: AUD {savings_from_shipping_cost_reduction:.2f}")
print(f"Net Impact (including all adjustments): AUD {net_impact:.2f}")


Total Shipping Cost in Non-Free Shipping Areas: AUD 5963.67
Reduced Shipping Cost in Non-Free Shipping Areas: AUD 5555.16
Savings from Shipping Cost Reduction: AUD 408.51
Net Impact (including all adjustments): AUD -4853.10


In [19]:
# Assuming all costs include a 10% GST
gst_rate = 0.10
price_increase_percentage = 6.85 / 100

# Remove GST from the subtotal to get the base amount
covered_orders['base_subtotal'] = covered_orders['adjusted_subtotal'] / (1 + gst_rate)

# Apply the price increase to the base amount
covered_orders['new_base_subtotal'] = covered_orders['base_subtotal'] * (1 + price_increase_percentage)

# Add GST back to the new base subtotal
covered_orders['new_adjusted_subtotal'] = covered_orders['new_base_subtotal'] * (1 + gst_rate)

# Calculate additional revenue from the new adjusted subtotal
additional_revenue_from_price_increase = covered_orders['new_adjusted_subtotal'].sum() - covered_orders['adjusted_subtotal'].sum()

# Similarly, adjust for non-free shipping areas if needed
non_free_shipping_areas['base_shipping'] = non_free_shipping_areas['shipping'] / (1 + gst_rate)
non_free_shipping_areas['new_base_shipping'] = non_free_shipping_areas['base_shipping'] * (1 - price_increase_percentage)
non_free_shipping_areas['new_shipping'] = non_free_shipping_areas['new_base_shipping'] * (1 + gst_rate)

# Calculate savings from reduced shipping costs
savings_from_shipping_cost_reduction = non_free_shipping_areas['shipping'].sum() - non_free_shipping_areas['new_shipping'].sum()

# Calculate the net impact considering all adjustments
net_impact = additional_revenue_from_price_increase + savings_from_shipping_cost_reduction - total_shipping_cost_uncovered_orders

# Print results
print(f"Additional Revenue from Price Increase: AUD {additional_revenue_from_price_increase:.2f}")
print(f"Savings from Shipping Cost Reduction: AUD {savings_from_shipping_cost_reduction:.2f}")
print(f"Net Impact (including all adjustments): AUD {net_impact:.2f}")


Additional Revenue from Price Increase: AUD 13924.47
Savings from Shipping Cost Reduction: AUD 408.51
Net Impact (including all adjustments): AUD -4853.10


C:\Users\olcay\AppData\Local\Temp\ipykernel_9108\3261991072.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covered_orders['base_subtotal'] = covered_orders['adjusted_subtotal'] / (1 + gst_rate)
C:\Users\olcay\AppData\Local\Temp\ipykernel_9108\3261991072.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covered_orders['new_base_subtotal'] = covered_orders['base_subtotal'] * (1 + price_increase_percentage)
C:\Users\olcay\AppData\Local\Temp\ipykernel_9108\3261991072.py:12: SettingWithCopyWarning: 
A val

# Old Strategy Weighted Average general Increase

In [ ]:
# Compute the weighted average of the required increase percentages
weighted_avg_required_increase_percentage = (
    (grouped_data['required_increase_percentage'] * grouped_data['adjusted_subtotal']).sum() /
    grouped_data['adjusted_subtotal'].sum()
)

print(weighted_avg_required_increase_percentage)

In [38]:

# Apply the weighted average increase to the adjusted subtotal
grouped_data['final_increased_subtotal_weighted'] = grouped_data['adjusted_subtotal'] * (1 + (weighted_avg_required_increase_percentage / 100))

# Calculate the taxes on the final increased subtotal
grouped_data['final_increased_taxes_weighted'] = grouped_data['final_increased_subtotal_weighted'] * 0.10

# Calculate the final increased total including taxes and shipping
grouped_data['final_increased_total_weighted'] = grouped_data['final_increased_subtotal_weighted'] + grouped_data['final_increased_taxes_weighted'] + grouped_data['shipping']

# Determine the coverage difference
grouped_data['coverage_difference_weighted'] = grouped_data['final_increased_total_weighted'] - grouped_data['original_total']
grouped_data['covers_shipping_weighted'] = grouped_data['coverage_difference_weighted'] >= 0

In [39]:
grouped_data.head(10)

,Name,subtotal,discount,shipping,original_total,province,adjusted_subtotal,required_increase_percentage,final_increased_subtotal_weighted,final_increased_taxes_weighted,final_increased_total_weighted,coverage_difference_weighted,covers_shipping_weighted
0,TRAU48673,1377.0,20.0,93.75,1450.75,Victoria,1357.0,6.908622,1450.962680,145.096268,1689.808948,239.058948,True
1,TRAU48674,109.0,0.0,12.29,121.29,Victoria,109.0,11.275229,116.547481,11.654748,140.492229,19.202229,True
2,TRAU48675,2440.0,0.0,154.02,2594.02,Queensland,2440.0,6.312295,2608.952793,260.895279,3023.868072,429.848072,True
3,TRAU48676,530.0,0.0,70.00,600.00,New South Wales,530.0,13.207547,566.698762,56.669876,693.368639,93.368639,True
4,TRAU48677,608.0,0.0,86.22,694.22,Victoria,608.0,14.180921,650.099712,65.009971,801.329683,107.109683,True
5,TRAU48678,294.0,0.0,56.89,350.89,Victoria,294.0,19.350340,314.357427,31.435743,402.683169,51.793169,True
6,TRAU48679,114.0,0.0,12.29,126.29,New South Wales,114.0,10.780702,121.893696,12.189370,146.373066,20.083066,True
7,TRAU48681,279.0,0.0,24.67,303.67,New South Wales,279.0,8.842294,298.318782,29.831878,352.820661,49.150661,True
8,TRAU48682,16.0,0.0,7.01,23.01,New South Wales,16.0,43.812500,17.107887,1.710789,25.828676,2.818676,True
9,TRAU48684,43.0,0.0,8.77,51.77,Victoria,43.0,20.395349,45.977447,4.597745,59.345191,7.575191,True


In [40]:
# Compute the weighted average of the required increase percentages
weighted_avg_required_increase_percentage = (
    (grouped_data['required_increase_percentage'] * grouped_data['adjusted_subtotal']).sum() /
    grouped_data['adjusted_subtotal'].sum()
)


In [41]:
# Apply the weighted average increase to the adjusted subtotal
grouped_data['final_increased_subtotal_weighted'] = grouped_data['adjusted_subtotal'] * (1 + (weighted_avg_required_increase_percentage / 100))

# Calculate the taxes on the final increased subtotal
grouped_data['final_increased_taxes_weighted'] = grouped_data['final_increased_subtotal_weighted'] * 0.10

# Calculate the final increased total including taxes and shipping
grouped_data['final_increased_total_weighted'] = grouped_data['final_increased_subtotal_weighted'] + grouped_data['final_increased_taxes_weighted'] + grouped_data['shipping']

# Calculate the original total (subtotal + shipping)
grouped_data['original_total'] = grouped_data['adjusted_subtotal'] + grouped_data['shipping']

# Determine the coverage difference
grouped_data['coverage_difference_weighted'] = grouped_data['final_increased_total_weighted'] - grouped_data['original_total']
grouped_data['covers_shipping_weighted'] = grouped_data['coverage_difference_weighted'] >= 0


In [42]:
# Filter rows where covers_shipping_weighted is False
false_covers_shipping = grouped_data[grouped_data['covers_shipping_weighted'] == True]

# Display the filtered rows
false_covers_shipping

,Name,subtotal,discount,shipping,original_total,province,adjusted_subtotal,required_increase_percentage,final_increased_subtotal_weighted,final_increased_taxes_weighted,final_increased_total_weighted,coverage_difference_weighted,covers_shipping_weighted
0,TRAU48673,1377.0,20.0,93.75,1450.75,Victoria,1357.0,6.908622,1450.962680,145.096268,1689.808948,239.058948,True
1,TRAU48674,109.0,0.0,12.29,121.29,Victoria,109.0,11.275229,116.547481,11.654748,140.492229,19.202229,True
2,TRAU48675,2440.0,0.0,154.02,2594.02,Queensland,2440.0,6.312295,2608.952793,260.895279,3023.868072,429.848072,True
3,TRAU48676,530.0,0.0,70.00,600.00,New South Wales,530.0,13.207547,566.698762,56.669876,693.368639,93.368639,True
4,TRAU48677,608.0,0.0,86.22,694.22,Victoria,608.0,14.180921,650.099712,65.009971,801.329683,107.109683,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
622,TRAU49344,1996.2,20.0,154.50,2130.70,Queensland,1976.2,7.818035,2113.037913,211.303791,2478.841705,348.141705,True
623,TRAU49345,178.2,0.0,17.11,195.31,Victoria,178.2,9.601571,190.539093,19.053909,226.703003,31.393003,True
624,TRAU49346,2281.1,0.0,30.16,2311.26,Australian Capital Territory,2281.1,1.322169,2439.050088,243.905009,2713.115097,401.855097,True
625,TRAU49347,601.2,0.0,37.29,638.49,New South Wales,601.2,6.202595,642.828860,64.282886,744.401746,105.911746,True


### Coverage difference

In [135]:
# Calculate the percentage reduction in shipping costs needed
percentage_reduction_in_shipping = abs(grouped_data[grouped_data['covers_shipping_weighted'] == False]['coverage_difference_weighted'].sum()) / grouped_data['shipping'].sum() * 100

# Calculate the new shipping costs after the reduction
grouped_data['reduced_shipping'] = grouped_data['shipping'] * (1 - percentage_reduction_in_shipping / 100)

# Calculate the new final total with reduced shipping costs
grouped_data['new_final_total'] = (grouped_data['adjusted_subtotal'] + grouped_data['reduced_shipping']) * (1 + weighted_avg_required_increase_percentage / 100)

# Calculate the new coverage difference with reduced shipping
grouped_data['new_coverage_difference'] = grouped_data['new_final_total'] - grouped_data['original_total']
grouped_data['new_covers_shipping'] = grouped_data['new_coverage_difference'] >= 0


In [136]:
# Sum the new total coverage difference and check the coverage rate
new_total_true_coverage_difference = grouped_data[grouped_data['new_covers_shipping'] == True]['new_coverage_difference'].sum()
new_total_false_coverage_difference = grouped_data[grouped_data['new_covers_shipping'] == False]['new_coverage_difference'].sum()

# # Export the final dataset
# final_data_path = '/path/to/save/final_data.xlsx'
# grouped_data.to_excel(final_data_path, index=False)
print(new_total_true_coverage_difference)
print(new_total_false_coverage_difference)

27527.044755805742
-4464.130785440026


In [137]:
grouped_data.head(5)

,Name,subtotal,discount,shipping,original_total,province,adjusted_subtotal,required_increase_percentage,final_increased_subtotal_weighted,final_increased_taxes_weighted,final_increased_total_weighted,coverage_difference_weighted,covers_shipping_weighted,reduced_shipping,new_final_total,new_coverage_difference,new_covers_shipping
0,TRAU48673,1377.0,20.0,93.75,1470.75,Victoria,1357.0,6.908622,1447.800271,144.780027,1686.330298,215.580298,True,93.277374,1547.319068,76.569068,True
1,TRAU48674,109.0,0.0,12.29,121.29,Victoria,109.0,11.275229,116.293463,11.629346,140.212809,18.922809,True,12.228042,129.339714,8.049714,True
2,TRAU48675,2440.0,0.0,154.02,2594.02,Queensland,2440.0,6.312295,2603.266515,260.326651,3017.613166,423.593166,True,153.243533,2766.763956,172.743956,True
3,TRAU48676,530.0,0.0,70.00,600.00,New South Wales,530.0,13.207547,565.463628,56.546363,692.009991,92.009991,True,69.647106,639.770997,39.770997,True
4,TRAU48677,608.0,0.0,86.22,694.22,Victoria,608.0,14.180921,648.682804,64.868280,799.771084,105.551084,True,85.785336,740.208251,45.988251,True


In [138]:
# Calculate the net difference
net_difference = new_total_true_coverage_difference - abs(new_total_false_coverage_difference)

# Output the net difference
print("Net Difference: ", net_difference)



Net Difference:  23062.913970365717


In [139]:
# Assuming grouped_data is your DataFrame

# Total True Coverage Difference Before Adjustment
total_true_coverage_difference_before = grouped_data[grouped_data['covers_shipping_weighted'] == True]['coverage_difference_weighted'].sum()

# Total False Coverage Difference Before Adjustment
total_false_coverage_difference_before = grouped_data[grouped_data['covers_shipping_weighted'] == False]['coverage_difference_weighted'].sum()

# Net Difference Before Adjustment
net_difference_before = total_true_coverage_difference_before - abs(total_false_coverage_difference_before)

# Output the results
print("Total True Coverage Difference Before Adjustment: ", total_true_coverage_difference_before)
print("Total False Coverage Difference Before Adjustment: ", total_false_coverage_difference_before)
print("Net Difference Before Adjustment: ", net_difference_before)


Total True Coverage Difference Before Adjustment:  72954.42651361092
Total False Coverage Difference Before Adjustment:  -163.53751361086597
Net Difference Before Adjustment:  72790.88900000005


In [140]:
total_true_coverage_difference_before = grouped_data[grouped_data['covers_shipping_weighted'] == True]['coverage_difference_weighted'].sum()
total_false_coverage_difference_before = grouped_data[grouped_data['covers_shipping_weighted'] == False]['coverage_difference_weighted'].sum()
net_difference_before = total_true_coverage_difference_before - abs(total_false_coverage_difference_before)
